<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
import pandas as pd

data = pd.read_csv("../datasets/market_insights_overview_all.csv")
data["date"] = pd.to_datetime(data["date"])
# data = pd.read_csv("../datasets/market_insights_overview_5p.csv")



FileNotFoundError: [Errno 2] No such file or directory: '../datasets/market_insights_overview_all.csv'

In [ ]:
data.head()
data.info()

In [ ]:
categories = pd.read_csv("../datasets/solr_categories_all_16_09_2021.csv")
categories.head()

In [ ]:
# Join market_insight table with category names
category_names = categories[[ "internal_doc_id", "title"]]

data_joined = pd.merge(data, category_names, how="left", left_on=["cat_id"], right_on=["internal_doc_id"])
data_joined = data_joined.rename(columns={"title": "cat_name"})
data_joined.head()

In [ ]:
print(category_names["title"])
categories_file = open("categories.txt", "w")
for cat_line in category_names["title"]:
    categories_file.write(cat_line)
    categories_file.write("\n")
categories_file.close()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Set pandas to use decimal insteaf of scientific e^x notaion
pd.set_option("display.float_format", lambda x: "%.0f" % x)

interesting_data = data_joined[["date", "hits", "clicks", "product_id", "cat_id", "cat_name" ]]

print(f"Unique products: {data['product_id'].nunique()}")
print(f"Unique categories: {data['cat_id'].nunique()}")


grouped = interesting_data.groupby(["cat_id", "date"], as_index=False).sum()

# Filter away early 2018 data as it seems to contain errors
grouped = grouped.loc[grouped['date'] > '2018-12-01']

print(grouped)
grouped.describe()

I perioden august 2018 til perioden august 2021 ble det registrert 1320 unike kategorier.
Et nevralnett med 1320 i bredde er litt stort kanskje? Det vil også låse modellen til X antall kategorier.
Dette er et argument for å se på hver kategori inviduelt og ha en egen modell per kategori.

Ulempen med det igjen er at det blir veldig mange modeller...

In [ ]:
print(grouped[grouped["hits"] == 1].count())
grouped.boxplot(column=["hits", "clicks"])
grouped.hist(column=["hits", "clicks"], range=(0, 50))

Vi ser at antall klikk for en kategori per dag avtar hurtig! Kan den følge en kjent sannsynlighet distribusjon?
F.eks: pareto distribution?

In [ ]:
import matplotlib.dates as mdate
date_locator = mdate.MonthLocator()
formatter = mdate.DateFormatter("%Y-%m")
all_categories = data["cat_id"].tolist()
for cat in all_categories[:15]:
    cat_name = categories.loc[categories["internal_doc_id"] == cat]["title"]
    grouped[grouped["cat_id"] == cat].plot(
        x="date", 
        y=["hits", "clicks"], 
        title=f"categori_id: {cat_name}", 
        figsize=(30, 10),
        )#.xaxis.set_major_locator(date_locator)
#grouped.plot(x="date", y=["hits", "clicks"],)
#interesting_data.plot()

In [ ]:
turmat = grouped[grouped["cat_id"] == 11488]
turmat.head()
name = categories.loc[categories["internal_doc_id"] == 11488]
name.head()
turmat.plot(
        x="date", 
        y=["hits", "clicks"], 
        title=f"categori: Turmat", 
        figsize=(30, 10),)

In [ ]:
vapenskap = grouped[grouped["cat_id"] == 12738]
vapenskap.head()
name = categories.loc[categories["internal_doc_id"] == 12738]
name.head()
vapenskap.plot(
        x="date", 
        y=["hits", "clicks"], 
        title=f"categori: Våpenskap", 
        figsize=(30, 10),)

In [ ]:
kopp_og_krus = grouped[grouped["cat_id"] == 12663]
kopp_og_krus.head()
name = categories.loc[categories["internal_doc_id"] == 12663]
name.head()
kopp_og_krus.plot(
        x="date", 
        y=["hits", "clicks"], 
        title=f"categori: Kopp og krus", 
        figsize=(30, 10),)

In [ ]:
Det virker ikke som om det er data på kopper og krus. Dette burde undersøkes nærmere. Forhåpentligvis finnes dataen, men den er ikke hentet ut og sendt til oss.

In [ ]:
hengekøye = grouped[grouped["cat_id"] == 11514]
hengekøye.head()
name = categories.loc[categories["internal_doc_id"] == 11514]
name.head()
hengekøye.plot(
        x="date", 
        y=["hits", "clicks"], 
        title=f"categori: Hengekøye", 
        figsize=(30, 10),)

In [ ]:
gpu = grouped[grouped["cat_id"] == 30]
gpu.head()
name = categories.loc[categories["internal_doc_id"] == 30]
name.head()
gpu.plot(
        x="date", 
        y=["hits", "clicks"], 
        title=f"categori: Hengekøye", 
        figsize=(30, 10),)

In [ ]:
import numpy as np
name.head()
import seaborn as sns
print(np.corrcoef(data["hits"], data["clicks"]))
correlation_matrix = data[["hits", "clicks"]].corr()
sns.heatmap(correlation_matrix)

Høy korrelasjon mellom hits og klicks!

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.hist(figsize=(20,15))

På de to siste grafene ser vi at de aller fleste dagene så har vi 0 clicks og hits.
Noen veldig få dager har vi 1 click / hit og enda færre dager har vi flere.